In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
from thesis_v2 import dir_dict
from thesis_v2.training.training_aux import load_training_results

In [3]:
from sys import path
from os.path import join, exists, dirname
from os import makedirs



In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200208 as param_iterator_obj,
    keygen
)

In [5]:
from numpy.linalg import norm
import numpy as np

In [6]:
from sklearn.decomposition import PCA
from pickle import dump

In [7]:


def calc_raw_stats(weight_matrix):
    # weight_matrix is N * D
    # N = number of filters
    # D = size of a (flattened) filter.
    mean_all = weight_matrix.mean(axis=1)
    std_all = weight_matrix.std(axis=1)
    print('mean_all', mean_all.mean(), mean_all.std())
    print('std_all', std_all.mean(), std_all.std())
    
def calc_normed_pca(weight_matrix, verbose=False):
    # here `normed` means PCA after normalization of weight_matrix
    
    # there are multiple ways to define `normalization`
    # 
    # a simple way is to make each flattened filter with unit norm.
    weight_matrix_norm = norm(weight_matrix, axis=1, keepdims=True)
#     assert weight_matrix_norm.shape == (256,1)
    weight_matrix_normed = weight_matrix/weight_matrix_norm
    
    # tested, works as expected.
    if verbose:
        assert np.allclose(norm(weight_matrix_normed, axis=1), 1.0)
    
    
    # then let's do PCA
    pca_obj = PCA(svd_solver='full')
    pca_obj.fit(weight_matrix_normed)
    if verbose:
        # top 10 explains 60%;
        # top 20 explains 85%;
        # looks fine to me, compared to https://doi.org/10.1101/677237
        # "Recurrent networks can recycle neural resources to flexibly trade speed for accuracy in visual recognition"
        
        print(np.cumsum(pca_obj.explained_variance_ratio_)[:20])
        print(pca_obj.components_.shape)
    
    # then let's return and save them.
    return {
        'weight_matrix_norm': weight_matrix_norm,
        'weight_matrix': weight_matrix,
        'components': pca_obj.components_,
        'explained_variance_ratio': pca_obj.explained_variance_ratio_,
    }

def construct_back_by_interleave(x1, x2, by1, by2):
    n1, m1 = x1.shape
    n2, m2 = x2.shape
    assert n1 % by1 == 0
    assert n2 % by2 == 0
    assert m1==m2 and n1//by1 == n2//by2
    
    x1 = x1.reshape(n1//by1, by1, m1)
    x2 = x2.reshape(n2//by2, by2, m2)
    return np.concatenate((x1,x2), axis=1).reshape((n1+n2, m1))


def save_one_result(filename, data):
    makedirs(dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        dump(data, f)

In [8]:
# create dir
global_vars = {
    'save_dir': join(dir_dict['analyses'],
                     'scripts+yuanyuan_8k_a_3day+maskcnn_polished_with_rcnn_k_bl+20200208',
                     'pca')
}

def prepare_dir():
    save_dir = global_vars['save_dir']
    if not exists(save_dir):
        makedirs(save_dir)
prepare_dir()

In [9]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [18]:
def collect_all():
    save_dir = global_vars['save_dir']
    count = 0
    verbose = True
    for idx, param in enumerate(param_iterator_obj().generate()):
        
        if param['rcnn_bl_cls'] <= 2:
            # ignore cls=1,2 models.
            continue
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        result = load_training_results(key, return_model=False, return_checkpoint=True)
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        # load loss curve to make sure we are good in approximation.
        corr_test = ((cc_raw/cc_max_all_neurons)**2).mean()
        # only keep 0.6+ models
        if corr_test < 0.6:
            continue
        
        
        count += 1        
        
        
        # 10 to go.
        
        def filename_gen(x):
            return join(save_dir, f'{idx}/unit_norm', f'{x}.pkl')
        
        filename_set = {'ff', 'lateral'}
        if all([exists(filename_gen(x)) for x in filename_set]):
            # right now if partially done, the partial results will be fully overwritten.
            continue
        
        # check layer one lateral.
        weight_np_ff = result['checkpoint']['model']['moduledict.bl_stack.layer_list.0.b_conv.weight'].numpy().astype(np.float64)
        weight_np_lateral = result['checkpoint']['model']['moduledict.bl_stack.layer_list.0.l_conv.weight'].numpy().astype(np.float64)

        kernel_shape = weight_np_ff.shape[2:]
        assert weight_np_ff.shape[2:] == weight_np_lateral.shape[2:] == kernel_shape
        assert weight_np_ff.ndim == weight_np_lateral.ndim == 4
        channel_low = weight_np_ff.shape[1]
        channel_high = weight_np_ff.shape[0]
        assert weight_np_lateral.shape[:2] == (channel_high, channel_high)
        
        weight_np_ff = weight_np_ff.reshape((channel_low*channel_high, kernel_shape[0]*kernel_shape[1]))
        weight_np_lateral = weight_np_lateral.reshape((channel_high*channel_high, kernel_shape[0]*kernel_shape[1]))
        
        
#         print(weight_np_lateral.shape, weight_np_ff.shape)
        
        if verbose:
            print('idx', idx)
            print('key', key)
            # show some raw stats.
            calc_raw_stats(weight_np_ff)
            calc_raw_stats(weight_np_lateral)
        pca_res_dict = dict()
        pca_res_dict['ff'] = calc_normed_pca(weight_np_ff, verbose)
        pca_res_dict['lateral'] = calc_normed_pca(weight_np_lateral, verbose)
        
        for fn in filename_set:
            save_one_result(filename_gen(fn), pca_res_dict[fn])
        
    print(count)


In [19]:
collect_all()

idx 4
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l17/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lmse/m_se0
mean_all 0.0021453514602308597 0.011041576935916838
std_all 0.03730825332084365 0.008581598569429815
mean_all -0.003758395509317841 0.018497340496927975
std_all 0.04503013349178181 0.018651283408947308
[0.25252757 0.43092877 0.59263994 0.69575447 0.76728898 0.82243533
 0.86496472 0.90578984 0.93908701 0.96109679 0.97847351 0.98783113
 0.99619327 0.99911707 1.         1.        ]
(16, 49)
[0.12416554 0.23888427 0.33571094 0.41867133 0.48779605 0.55196417
 0.60783721 0.65532422 0.69737152 0.73072251 0.76031098 0.78746884
 0.80880883 0.8263393  0.84293429 0.85850483 0.87262839 0.88583998
 0.89739023 0.90635018]
(49, 49)
idx 5
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l17/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_accinstant/

idx 196
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l2/k_l17/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0010421274492452793 0.011419393890034433
std_all 0.026052060257350202 0.006627614624522618
mean_all -0.0024848936995772847 0.018108669990626727
std_all 0.029621000826984475 0.0128025772816328
[0.2410206  0.44344068 0.59679843 0.70374612 0.78201035 0.84549769
 0.87471829 0.90312397 0.92585558 0.9415438  0.95595356 0.96800523
 0.97641318 0.98394622 0.98948977 0.99392593 0.99612116 0.99733051
 0.99816574 0.99868414]
(48, 49)
[0.18952761 0.32857825 0.4408283  0.53984584 0.61499036 0.67278609
 0.71241781 0.74647956 0.77422307 0.7943101  0.81058456 0.82668069
 0.84264102 0.85410278 0.86356566 0.87200838 0.88014368 0.88746551
 0.89411659 0.90011738]
(49, 49)
idx 197
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l2/k_l17/k_p3/ptavg/bn_a_fcFa

[0.21319254 0.36502985 0.49160846 0.59428875 0.68455604 0.74748472
 0.78964779 0.82809292 0.86139653 0.89041231 0.91243104 0.93349716
 0.94687724 0.95925539 0.96921299 0.977585   0.98322693 0.98860202
 0.99255454 0.99440149]
(48, 81)
[0.16666453 0.29671485 0.4117213  0.50361376 0.57115272 0.63491532
 0.67439964 0.70899342 0.73665353 0.75577521 0.77422346 0.79009575
 0.80497086 0.81533822 0.82430022 0.83195743 0.83945691 0.84648282
 0.85288102 0.8578482 ]
(81, 81)
idx 358
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l113/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c4/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lmse/m_se0
mean_all 0.00023823779783559759 0.006817445223523504
std_all 0.03067765007786392 0.004014869277013418
mean_all -0.002165762575398521 0.013732926593710981
std_all 0.03448191966813178 0.010897473547744759
[0.17737568 0.30186268 0.39758126 0.48507646 0.55034903 0.60990494
 0.66057801 0.70210014 0.73792069 0.77040126 0.80139736 0

idx 548
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l113/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lpoisson/m_se0
mean_all -0.006834946337810769 0.013449163609602837
std_all 0.04931894319932847 0.016392201556880613
mean_all -0.0038413412557791104 0.024823764565898295
std_all 0.04582834902285002 0.01576189089551004
[0.26703453 0.43319868 0.55815607 0.6690117  0.74856709 0.81510321
 0.8637031  0.89793083 0.92544342 0.94883367 0.96914565 0.97972821
 0.99000376 0.99550729 1.         1.        ]
(16, 169)
[0.2117589  0.34048087 0.44393006 0.51764936 0.57610252 0.62110796
 0.66263008 0.69301317 0.72178333 0.74739199 0.7682546  0.78584045
 0.80101989 0.81424231 0.82594362 0.83650308 0.84646534 0.85582493
 0.864116   0.87090226]
(169, 169)
idx 580
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l2/k_l17/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r

[0.2054124  0.34981635 0.4583502  0.54367762 0.60918567 0.66864316
 0.72059144 0.75773254 0.79003726 0.82053751 0.84462002 0.86738526
 0.88762264 0.90290565 0.91645539 0.92906673 0.94078585 0.95214323
 0.96250129 0.96940312]
(48, 169)
[0.13695066 0.23862398 0.32985325 0.41401075 0.48042067 0.53874728
 0.58673966 0.63175547 0.66941649 0.70278094 0.72985706 0.75497117
 0.77928169 0.79701062 0.8115175  0.8251802  0.8380842  0.85011758
 0.86191957 0.87014344]
(169, 169)
idx 676
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l17/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lpoisson/m_se0
mean_all -0.0018114521425382259 0.012576497887255633
std_all 0.02630141035480969 0.005549629602930462
mean_all -0.0032569859084448875 0.017826607143482477
std_all 0.03235182992519288 0.01415476238727058
[0.23866527 0.4632198  0.63242121 0.73831312 0.79569179 0.84543795
 0.88331812 0.91140926 0.93562968 0.9536667  0.9649

...

idx 2596
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l17/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lmse/m_se1
mean_all -0.0068900332233334 0.01950076688704077
std_all 0.02865339590851966 0.010083221508572144
mean_all -0.00265604455228213 0.022155192323919937
std_all 0.03890119593137961 0.019704608816411482
[0.4457741  0.61759675 0.76761183 0.8164341  0.86146265 0.90018102
 0.92283705 0.94356471 0.96211922 0.97684543 0.98622321 0.99264704
 0.99567057 0.99749798 0.99865027 0.99916757 0.99943305 0.9996152
 0.9997423  0.99983682]
(48, 49)
[0.19373953 0.35122635 0.46031823 0.56509367 0.63879407 0.70381204
 0.74897052 0.783677   0.8100929  0.83349192 0.85558032 0.87267684
 0.88688139 0.89654658 0.90420049 0.91165181 0.91837059 0.92488705
 0.93022613 0.9349379 ]
(49, 49)
idx 2597
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l17/k_p3/ptavg/bn_a_fcFal

mean_all -0.0023462042064102562 0.01951735399629942
std_all 0.043326008212632425 0.0142891122790173
[0.30807789 0.47504696 0.61371958 0.70144273 0.77612833 0.83343249
 0.87900377 0.91764231 0.94682685 0.96952702 0.98513209 0.99306082
 0.99658122 0.99879671 1.         1.        ]
(16, 81)
[0.17413041 0.30586327 0.39784879 0.4741487  0.5423258  0.5994452
 0.64551928 0.68565266 0.72104309 0.75591641 0.7815803  0.80308584
 0.82145076 0.8370919  0.8520786  0.86620991 0.87970287 0.89079294
 0.89979624 0.90765653]
(81, 81)
idx 2757
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l113/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_accinstant/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.0009715729344066955 0.004861189628878595
std_all 0.042274328881330384 0.007444082229228654
mean_all -0.0003955408575360285 0.01585951819447998
std_all 0.03950534547393747 0.015010189766043774
[0.29939535 0.45990961 0.6054249  0.69761695 0.76258908 0.81

idx 2852
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l113/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.000557486281916253 0.00846336249022204
std_all 0.04555126579102434 0.006591480740916056
mean_all -0.0011358079427785492 0.015017698656208013
std_all 0.047124694512668465 0.01448994429678795
[0.21684394 0.40353548 0.5680664  0.65692905 0.7298939  0.78932048
 0.84409289 0.88666505 0.91822432 0.943282   0.96429941 0.97702031
 0.98856367 0.99502337 1.         1.        ]
(16, 169)
[0.10876421 0.20113846 0.26786243 0.3281971  0.38611838 0.43653126
 0.48217807 0.52314989 0.55601724 0.58798339 0.61695408 0.64305458
 0.66847411 0.69148393 0.7121131  0.73227966 0.74892141 0.76529007
 0.77936716 0.7926433 ]
(169, 169)
idx 2853
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l113/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_

[0.22834503 0.35655542 0.47660087 0.57967304 0.64158121 0.68639846
 0.72392217 0.75277226 0.77909172 0.80120605 0.81966342 0.83637052
 0.84766923 0.8563624  0.86362202 0.86970085 0.87520516 0.88040373
 0.88526939 0.88997154]
(81, 81)
idx 2948
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l2/k_l113/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/sc0.01/sm0.000005/lpoisson/m_se1
mean_all 0.00019217461346918925 0.010193051191561992
std_all 0.03964957875075822 0.00714613417990827
mean_all -0.0010106791694964414 0.020044096338088
std_all 0.03683957662240399 0.016692146241561488
[0.2417773  0.37773944 0.47556408 0.5647209  0.64648887 0.70268691
 0.75154346 0.79198902 0.82340812 0.84992478 0.87454873 0.89368259
 0.91136182 0.92811759 0.94041031 0.95003086 0.95923076 0.96806989
 0.97513588 0.98071443]
(48, 169)
[0.17064571 0.28010335 0.36957044 0.44496037 0.50365005 0.55235793
 0.59751003 0.63648072 0.66871075 0.69615069 0.721

idx 3015
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_accinstant/sc0.01/sm0.000005/lpoisson/m_se1
mean_all 0.00345808442922377 0.012724781672583238
std_all 0.02775002297552488 0.006663503653109845
mean_all -0.0017892623995307219 0.015523476277785084
std_all 0.030428161314623597 0.011411045705812525
[0.31871999 0.5156803  0.66444364 0.74237508 0.80270583 0.8502919
 0.88465838 0.90652152 0.92733541 0.94387496 0.95916497 0.97181429
 0.98188906 0.98848578 0.99367118 0.99602797 0.99724805 0.99820292
 0.99888121 0.99921629]
(48, 81)
[0.17604023 0.30931151 0.42468864 0.5248066  0.5917038  0.646529
 0.68967035 0.7256748  0.75370337 0.77887216 0.80242486 0.82089227
 0.83636522 0.84673725 0.85667966 0.86489059 0.87223147 0.87878887
 0.88475661 0.88976474]
(81, 81)
idx 3044
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch48/num_l3/k_l113/k_p3/ptavg/bn_